In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
from datetime import datetime, timedelta
import os
import pytz

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

from sustaingym.data.load_moer import load_monthly_moer


In [ ]:
# moer_data_dir = 'sustaingym/data/moer_data'
moer_data_dir = 'sustaingym/data/moer/'
cnt = 0

minr, minf = 100., 100.
maxr, maxf = -100., 100.

for file in os.listdir(moer_data_dir):
    print(file)
    cnt += 1

    file_path = os.path.join(moer_data_dir, file)
    df = pd.read_csv(file_path,
                    compression='gzip',
                    index_col='time')
    df.index = pd.to_datetime(pd.DatetimeIndex(df.index))

    # check if any null values exist
    if len(df[df.isnull().any(axis=1)]) > 0:
        print("Null value")
        display(df[df.isnull().any(axis=1)])
    
    # check sorting order
    if df.index[0] < df.index[1]:
        print("Not sorted descending")
        display(df.head())

    minr = min(minr, np.min(df.min()))
    maxr = max(minr, np.max(df.max()['moer']))

print('mins: ', minr)
print('maxs: ', maxr)
print("total count: ", cnt)
# show example
display(df)

In [ ]:
AM_LA = pytz.timezone('America/Los_Angeles')

def plot_moer(day: datetime, ba: str, save_dir: str, forecast_freq=36):
    df = load_monthly_moer(day.year, day.month, ba, save_dir)
    df = df[(day <= df.index) & (df.index <= day + timedelta(days=1, seconds=600))]
    df.sort_index(inplace=True)

    # every so often, plot emissions forecast
    for i in range(0, len(df['moer']), forecast_freq):
        forecasts = df.iloc[i].drop('moer').values
        indices = df.iloc[i+1:i+1+len(forecasts)].index
        if i + len(forecasts) + 1> len(df['moer']):
            continue
        
        if i == 0:
            plt.plot(indices, forecasts, color='red', alpha=0.3, label='forecasts')
        else:
            plt.plot(indices, forecasts, color='red', alpha=0.3)

    dt_str = day.strftime('%Y-%m-%d')
    plt.title(f'MOER vs. Forecasted MOER: {dt_str}')
    plt.plot(df['moer'], linewidth=2, label='actual')
    plt.xlabel('time (UTC)')
    plt.ylabel('MOER (kg CO2 / kWh)')
    plt.xticks(rotation=30)
    plt.legend()

plot_moer(datetime(2021, 6, 3, tzinfo=AM_LA), 'SGIP_CAISO_SCE', moer_data_dir)

In [ ]:
plot_moer(datetime(2019, 6, 15, tzinfo=AM_LA), 'SGIP_CAISO_SCE', moer_data_dir)